This notebook is used to produce Figures 7-9 in the notes: steady-state solutions for some different parameter choices.
We are just computing the steady elevation solutions via $\pmb{y}_e = -\mathsf{A}^{-1}\pmb{b}$ (see previous notebook or the notes), and using these to compute the horizontal velocity perturbation solutions.

In [ ]:
# add path to code
import sys
sys.path.insert(0, '../code')
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.fft import ifft2,fft2
from operators import compute_h, compute_s,h_steady,s_steady,compute_u,compute_v,compute_w
from params import *

Define the melt-rate perturbation:

In [ ]:
sigma = (L/4)/3                                                    # standard deviation for gaussian melt anomaly

# For Figure 8 (melting) set:
m = 5*np.exp(-0.5*(sigma**(-2))*(x**2+y**2 ))*(t_r/3.154e7)/H     # melt anomaly with amplitude of 5 m/yr

# For Figure 9 and 10 set the negative version (freeze-on):
# m = 5*np.exp(-0.5*(sigma**(-2))*(x**2+y**2 ))*(t_r/3.154e7)/H     # melt anomaly with amplitude of 5 m/yr
# **(also setting L=4 in params.py for figure 10)

# print('{:2e}'.format(np.max(np.abs(m))))

Define a vertical exaggeration for plotting the elevations (since these ones are a small fraction of the ice thickness):

In [ ]:
ex = 5e1   # vertical exaggeration factor
if L<10:
    ex = 3e1  # set to smaller value for the example in figure 10.

First we compute the solutions with a non-advective background state ($\alpha=0$). The only annoying thing here is interpolating the velocities onto the deformed grid defined by $z\mapsto (1-z)\gamma s + z\gamma h $ (for plotting), where $\gamma$ is the vertical exaggeration factor.

In [ ]:
# First compute the elevation solutions

alpha = 0
h_e = h_steady(k,alpha,m)
s_e = s_steady(k,alpha,m)

# extract the centerline profiles 
yc = int(Ny/2.)     # y index at centerline 
tc = -1             # last timestep (steady profile)

he_c = h_e[tc,:,yc] 
se_c = s_e[tc,:,yc]

# Next we compute the velocity fields at a few depths:

Nz = 21
z_plt = np.linspace(0,1,num=Nz)
U = np.zeros((Nt,Nx,Ny,Nz))
W = np.zeros((Nt,Nx,Ny,Nz))

for i in range(Nz):
    U[:,:,:,i] = compute_u(h_e,s_e,k,kx,z_plt[i])
    W[:,:,:,i] = compute_w(h_e,s_e,k,z_plt[i])

step = 1
x_plt = x0
Ue_c = U[tc,:,yc,:]
We_c = W[tc,:,yc,:]

s_plt = se_c
h_plt = he_c


# Next we interpolate the velocity solutions on the deformed grid for plotting

x_g,z_g = np.meshgrid(x_plt,z_plt)

z_d = (1-z_g)*ex*s_plt + z_g*(ex*h_plt+1)

nxi = 101
nzi = 101

xi = np.linspace(x_g.min(),x_g.max(),num=nxi)
zi = np.linspace(z_d.min(),z_d.max(),num=nzi)

Xi,Zi = np.meshgrid(xi,zi)

points = (x_g.flatten(),z_d.flatten())
points_i = (Xi,Zi)

U_plt = griddata(points=points,values=Ue_c.T.flatten(), xi=points_i,fill_value=0)
W_plt = griddata(points=points,values=We_c.T.flatten(), xi=points_i,fill_value=0)

speed = np.sqrt(U_plt**2 + W_plt**2)
max_speed = np.max(speed)

# # Print some sanity-check information:
# print('With the scales in params.py:')
# print('max u = '+str(np.max(np.abs(U_plt))*3.154e7*H/t_r)+' m/yr')
# print('max w = '+str(np.max(np.abs(W_plt))*3.154e7*H/t_r)+' m/yr')
# print('max h = '+str(np.max(np.abs(h_plt))*H)+' m')
# print('max s = '+str(np.max(np.abs(s_plt))*H)+' m')

# # Sanity check plot to make sure interpolation on regular grid (from deformed grid) works: 
# plt.close()
# plt.subplot(211)
# plt.contourf(x_g,z_d,We_c.T*H*3.154e7/t_r,cmap='Blues')
# #plt.plot(x_g.flatten(),z_d.flatten(),'ko',markersize=0.25)
# plt.colorbar()
# plt.subplot(212)
# plt.contourf(xi,zi,W_plt*H*3.154e7/t_r,cmap='Blues',extend='both')
# plt.plot(x0,ex*he_c+1,linewidth=2,color='k',linestyle='--')
# plt.plot(x0,ex*se_c,linewidth=2,color='k',linestyle='--')
# plt.colorbar()
# plt.show()

Repeat the above steps for a positive value of the background advection parameter:

In [ ]:
alpha = 2e-1
h_e = h_steady(k,alpha,m)
s_e = s_steady(k,alpha,m)

# extract the centerline profiles 
yc = int(Ny/2.)     # y index at centerline 
tc = -1             # last timestep (steady profile)

he_c = h_e[tc,:,yc] 
se_c = s_e[tc,:,yc]

# Next we compute the velocity fields at a few depths:

Nz = 21
z_plt = np.linspace(0,1,num=Nz)
U = np.zeros((Nt,Nx,Ny,Nz))
W = np.zeros((Nt,Nx,Ny,Nz))

for i in range(Nz):
    U[:,:,:,i] = compute_u(h_e,s_e,k,kx,z_plt[i])
    W[:,:,:,i] = compute_w(h_e,s_e,k,z_plt[i])

step = 1
x_plt = x0
Ue_c = U[tc,:,yc,:]
We_c = W[tc,:,yc,:]

s_plt_2 = se_c
h_plt_2 = he_c


# Next we interpolate the velocity solutions on the deformed grid for plotting

x_g,z_g = np.meshgrid(x_plt,z_plt)

z_d = (1-z_g)*ex*s_plt + z_g*(ex*h_plt+1)

nxi = 101
nzi = 101

xi = np.linspace(x_g.min(),x_g.max(),num=nxi)
zi = np.linspace(z_d.min(),z_d.max(),num=nzi)

Xi,Zi = np.meshgrid(xi,zi)

points = (x_g.flatten(),z_d.flatten())
points_i = (Xi,Zi)

U_plt_2 = griddata(points=points,values=Ue_c.T.flatten(), xi=points_i,fill_value=0)
W_plt_2 = griddata(points=points,values=We_c.T.flatten(), xi=points_i,fill_value=0)

speed_2 = np.sqrt(U_plt_2**2 + W_plt_2**2)
max_speed_2 = np.max(speed_2)

Plot both solutions:

In [ ]:
plt.close()
fig = plt.figure(figsize=(14,4))

plt.subplot(121)
plt.title(r'(a) $\alpha=0$',fontsize=20,loc='left',zorder=100)
plt.plot(x_plt,ex*s_plt,linewidth=2.5,color='darkblue',zorder=16,label=r'$\gamma s$')
plt.plot(x_plt,ex*h_plt+1,linewidth=2.5,color='forestgreen',zorder=13,label=r'$1+\gamma h$')

plt.plot(x_plt,ex*(-delta*s_plt)+1,linewidth=2.5,color='k',linestyle='--',zorder=80)

plt.fill_between(x_plt,y1=ex*h_plt+1, y2=2*np.ones(np.size(x_plt)),facecolor='w',alpha=1.0,zorder=12)
plt.fill_between(x_plt,y1=ex*s_plt, y2=ex*h_plt+1,facecolor='aliceblue',alpha=1.0)
plt.fill_between(x_plt,y1=-2*np.ones(np.size(x_plt)), y2=ex*s_plt,facecolor='lightsteelblue',alpha=1,zorder=15)
stream = plt.streamplot(xi,zi,U_plt/max_speed,W_plt/max_speed,color=speed/max_speed,arrowsize=1.5,linewidth=2,minlength=0.6,density=2,cmap='Reds')

if L >10:
    plt.annotate(xy=(-L+1,1.03),text='air',fontsize=20,zorder=30)
    plt.annotate(xy=(-L+1,0.03),text='ice',fontsize=20,zorder=31)
    plt.annotate(xy=(-L+1,-0.15),text='water',fontsize=20,zorder=32)
else:
    plt.annotate(xy=(-L+0.1,1.03),text='air',fontsize=20,zorder=30)
    plt.annotate(xy=(-L+0.1,0.03),text='ice',fontsize=20,zorder=31)
    plt.annotate(xy=(-L+0.1,-0.15),text='water',fontsize=20,zorder=32)
    
plt.ylabel(r'$z$',fontsize=20)
plt.xlabel(r'$x$',fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(-0.5,1.25)
plt.xlim(-L,L)

plt.subplot(122)
plt.title(r'(b) $\alpha={1}/{5}$',fontsize=20,loc='left',zorder=100)

plt.plot(x_plt,ex*s_plt_2,linewidth=2.5,color='darkblue',zorder=16,label=r'$\gamma s$')
plt.plot(x_plt,ex*h_plt_2+1,linewidth=2.5,color='forestgreen',zorder=13,label=r'$1+\gamma h$')

# # vv hydrostatic thickness profile vv
plt.plot(x_plt,ex*(-delta*s_plt_2)+1,linewidth=2.5,color='k',linestyle='--',zorder=80,label=r'$1-\gamma\delta s$')

plt.fill_between(x_plt,y1=ex*h_plt_2+1, y2=2*np.ones(np.size(x_plt)),facecolor='w',alpha=1.0,zorder=12)
plt.fill_between(x_plt,y1=ex*s_plt_2, y2=ex*h_plt_2+1,facecolor='aliceblue',alpha=1.0)
plt.fill_between(x_plt,y1=-2*np.ones(np.size(x_plt)), y2=ex*s_plt_2,facecolor='lightsteelblue',alpha=1,zorder=15)
stream = plt.streamplot(xi,zi,U_plt_2/max_speed_2,W_plt_2/max_speed_2,color=speed_2/max_speed_2,arrowsize=1.5,linewidth=2,minlength=0.6,density=2,cmap='Reds')
plt.legend(fontsize=20,ncol=3,bbox_to_anchor=(0.55,-0.175))
plt.xlabel(r'$x$',fontsize=20)
plt.xticks(fontsize=16)
plt.gca().yaxis.set_ticklabels([])
plt.ylim(-0.5,1.25)
plt.xlim(-L,L)

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.12, 0.02, 0.75])
cbar = fig.colorbar(stream.lines,cax=cbar_ax,orientation='vertical')
cbar.ax.tick_params(labelsize=16)
cbar.set_label(r'$|\mathbf{u}|\,/\, \Vert\mathbf{u}\Vert_\infty$',fontsize=24,labelpad=10)
cbar.ax.tick_params(labelsize=16)

plt.savefig('profiles',bbox_inches='tight')
plt.show()
plt.close()

Yay! Notes: the dashed line at the upper surface is the perfect flotation thickness and the flow speed has been normalized by its maximum absolute value.